In [1]:
import pandas as pd

In [2]:
covid19DF = pd.read_csv("./Data/04-12-2020.csv", usecols = ["Admin2", "Province_State", "Country_Region",
                                                           "Confirmed"])

In [3]:
covid19DF_us = covid19DF.loc[covid19DF['Country_Region'] == 'US']
covid19DF_georgia = covid19DF_us.loc[covid19DF['Province_State'] == 'Georgia']
covid19DF_georgia

,Admin2,Province_State,Country_Region,Confirmed
65,Appling,Georgia,US,9
87,Atkinson,Georgia,US,2
99,Bacon,Georgia,US,14
101,Baker,Georgia,US,14
103,Baldwin,Georgia,US,52
...,...,...,...,...
2631,Whitfield,Georgia,US,24
2638,Wilcox,Georgia,US,22
2639,Wilkes,Georgia,US,6
2642,Wilkinson,Georgia,US,11


In [4]:
census_regions = pd.read_csv("./Data/census_georgia.csv")

In [5]:
for county in census_regions['Admin2']:
    census_regions.Admin2[census_regions.Admin2 == county] = county.replace(".", '').split()[0]

census_regions

,Admin2,2019
0,Appling,"18,386"
1,Atkinson,"8,165"
2,Bacon,"11,164"
3,Baker,"3,038"
4,Baldwin,"44,890"
...,...,...
154,Whitfield,"104,628"
155,Wilcox,"8,635"
156,Wilkes,"9,777"
157,Wilkinson,"8,954"


In [6]:
dijkstraDF = covid19DF_georgia.merge(census_regions, on="Admin2", how = 'inner')

In [7]:
dijkstraDF = dijkstraDF.drop('Province_State', 1).drop('Country_Region', 1)
dijkstraDF

,Admin2,Confirmed,2019
0,Appling,9,"18,386"
1,Atkinson,2,"8,165"
2,Bacon,14,"11,164"
3,Baker,14,"3,038"
4,Baldwin,52,"44,890"
...,...,...,...
149,Whitfield,24,"104,628"
150,Wilcox,22,"8,635"
151,Wilkes,6,"9,777"
152,Wilkinson,11,"8,954"


In [8]:
dijkstraDF.to_csv('dijkstraInput.csv')

In [9]:
counties_info = pd.read_csv(".//Counties_Info.csv")

In [10]:
counties_info.head()

,Source,Destination,Time,Confirmed,2019 Census
0,Clayton,Dekalb,26,869,"292,256"
1,Clayton,Fayette,22,99,"114,421"
2,Clayton,Fulton,26,1467,"1,063,937"
3,Clayton,Henry,27,260,"234,561"
4,Cobb,Douglas,30,153,"146,343"


In [11]:
import math
Density = []
for confirmed, census in zip(counties_info['Confirmed'], counties_info['2019 Census']):
    Density.append(math.ceil((confirmed/int(census.replace(',', ''))) * 1000))

In [12]:
counties_info['Density'] = Density


In [13]:
Density_Time = []
for density, time1 in zip(counties_info['Time'], counties_info['Density']):
    Density_Time.append(density* time1)

In [14]:
counties_info['Density * Time'] = Density_Time

In [15]:
counties_info

,Source,Destination,Time,Confirmed,2019 Census,Density,Density * Time
0,Clayton,Dekalb,26,869,"292,256",3,78
1,Clayton,Fayette,22,99,"114,421",1,22
2,Clayton,Fulton,26,1467,"1,063,937",2,52
3,Clayton,Henry,27,260,"234,561",2,54
4,Cobb,Douglas,30,153,"146,343",2,60
5,Cobb,Fulton,26,1467,"1,063,937",2,52
6,Coweta,Fayette,26,99,"114,421",1,26
7,Coweta,Fulton,48,1467,"1,063,937",2,96
8,Dekalb,Clayton,27,362,"292,256",2,54
9,Dekalb,Gwinnett,26,690,"936,250",1,26


In [16]:
dijkstra_input = []
cost_input = {}
for src, dest, dt, time in zip(counties_info['Source'], counties_info['Destination'], counties_info['Density * Time'], counties_info['Time']):
    dijkstra_input.append((src, dest, dt))
    cost_input.update({src+dest: time})
  

In [17]:
dijkstra_input

[('Clayton', 'Dekalb', 78),
 ('Clayton', 'Fayette', 22),
 ('Clayton', 'Fulton', 52),
 ('Clayton', 'Henry', 54),
 ('Cobb', 'Douglas', 60),
 ('Cobb', 'Fulton', 52),
 ('Coweta', 'Fayette', 26),
 ('Coweta', 'Fulton', 96),
 ('Dekalb', 'Clayton', 54),
 ('Dekalb', 'Gwinnett', 26),
 ('Dekalb', 'Fulton', 48),
 ('Dekalb', 'Henry', 78),
 ('Douglas', 'cobb', 30),
 ('Douglas', 'Fulton', 60),
 ('Fulton', 'Clayton', 52),
 ('Fulton', 'Cobb', 26),
 ('Fulton', 'Coweta', 48),
 ('Fulton', 'Dekalb', 72),
 ('Fulton', 'Douglas', 60),
 ('Fulton', 'Fayette', 42),
 ('Fulton', 'Gwinnett', 32),
 ('Gwinnett', 'Dekalb', 78),
 ('Gwinnett', 'Fulton', 64),
 ('Fayette', 'Clayton', 44),
 ('Fayette', 'Fulton', 84),
 ('Fayette', 'Coweta', 26),
 ('Henry', 'Clayton', 54),
 ('Henry', 'Dekalb', 117)]

In [18]:
from collections import deque, namedtuple


# we'll use infinity as a default distance to nodes.
inf = float('inf')
Edge = namedtuple('Edge', 'start, end, cost')


def make_edge(start, end, cost=1):
      return Edge(start, end, cost)


class Graph:
    def __init__(self, edges):
        # let's check that the data is right
        wrong_edges = [i for i in edges if len(i) not in [2, 3]]
        if wrong_edges:
            raise ValueError('Wrong edges data: {}'.format(wrong_edges))

        self.edges = [make_edge(*edge) for edge in edges]

    @property
    def vertices(self):
        return set(
            sum(
                ([edge.start, edge.end] for edge in self.edges), []
            )
        )

    def get_node_pairs(self, n1, n2, both_ends=True):
        if both_ends:
            node_pairs = [[n1, n2], [n2, n1]]
        else:
            node_pairs = [[n1, n2]]
        return node_pairs

    def remove_edge(self, n1, n2, both_ends=True):
        node_pairs = self.get_node_pairs(n1, n2, both_ends)
        edges = self.edges[:]
        for edge in edges:
            if [edge.start, edge.end] in node_pairs:
                self.edges.remove(edge)

    def add_edge(self, n1, n2, cost=1, both_ends=True):
        node_pairs = self.get_node_pairs(n1, n2, both_ends)
        for edge in self.edges:
            if [edge.start, edge.end] in node_pairs:
                return ValueError('Edge {} {} already exists'.format(n1, n2))

        self.edges.append(Edge(start=n1, end=n2, cost=cost))
        if both_ends:
            self.edges.append(Edge(start=n2, end=n1, cost=cost))

    @property
    def neighbours(self):
        neighbours = {vertex: set() for vertex in self.vertices}
        for edge in self.edges:
            neighbours[edge.start].add((edge.end, edge.cost))

        return neighbours

    def dijkstra(self, source, dest):
        assert source in self.vertices, 'Such source node doesn\'t exist'
        distances = {vertex: inf for vertex in self.vertices}
        previous_vertices = {
            vertex: None for vertex in self.vertices
        }
        distances[source] = 0
        vertices = self.vertices.copy()

        while vertices:
            current_vertex = min(
                vertices, key=lambda vertex: distances[vertex])
            vertices.remove(current_vertex)
            if distances[current_vertex] == inf:
                break
            for neighbour, cost in self.neighbours[current_vertex]:
                alternative_route = distances[current_vertex] + cost
                if alternative_route < distances[neighbour]:
                    distances[neighbour] = alternative_route
                    previous_vertices[neighbour] = current_vertex

        path, current_vertex = deque(), dest
        while previous_vertices[current_vertex] is not None:
            path.appendleft(current_vertex)
            current_vertex = previous_vertices[current_vertex]
        if path:
            path.appendleft(current_vertex)
        return path


graph = Graph(dijkstra_input)


In [19]:
print(graph.dijkstra("Fayette", "Dekalb"))

deque(['Fayette', 'Clayton', 'Dekalb'])


In [20]:
dijkstra_input_deadline = []

for src, dest, dt in zip(counties_info['Source'], counties_info['Destination'], counties_info['Time']):
    dijkstra_input_deadline.append((src, dest, dt))
  

In [26]:
def getPathCost(path):
    cost = 0
    v1 = path.popleft()
    while(len(path) > 0):
        v2 = path.popleft()
        cost = cost +  cost_input[v1+v2]
        v1 = v2;
    return cost;

def getPath(path):
    path_value = ""
    
    for node in path:
        path_value += (node + " ")
    return path_value
    
def find_safest_path(source, destination, deadline):
    if source == destination:
        print( "You are already at " + destination)
        return

    graph = Graph(dijkstra_input)
    path = graph.dijkstra(source, destination)
    path_value = getPath(path)
    cost = getPathCost(path)
    if deadline > 0 and deadline > cost:
        print(" The safest path is ", path_value ," with cost ", cost)
    elif deadline > 0 and deadline < cost:
        graph = Graph(dijkstra_input_deadline)
        path = graph.dijkstra(source, destination)
        path_value = getPath(path)
        cost = getPathCost(path)

        if cost > deadline:
            print(" There is no path that meets your deadline of ", deadline, "minutes. But, an alternate route takes you ", cost, " minutes")
        else:
            print(" The safest path that meets your", deadline ," minutes condition is ", path_value)
    else:
        print("The safest path from ", source, " to ", destination, " is ", path_value )



In [30]:
# for deadline condition replace the third arg with a positive value representing deadline
# if -ve value given, deadline condition is ignored.
find_safest_path("Coweta","Gwinnett",-1)

The safest path from  Coweta  to  Gwinnett  is  Coweta Fulton Gwinnett 
